# 4.2 - Office finder mongo version

#### Encontramos tu oficina

In [1]:
%pip install folium

Note: you may need to restart the kernel to use updated packages.


In [2]:
# librerias

from pymongo import MongoClient

import pandas as pd

import folium   # esto es para hacer mapas

import re

In [6]:
# conexion a mongo

cliente=MongoClient()

db=cliente.companies

colec=db.companies

In [10]:
query={'$and': [
    
    {'$or': [{'category_code': 'games_video'}, 
             {'category_code': 'web'},
             {'category_code': 'ecommerce'}]},
    
    {'founded_year': {'$gte': 1999}},
    {'deadpooled_year': None},
    {'number_of_employees': {'$ne': None}},
    {'number_of_employees': {'$ne': 0}},
    {'total_money_raised': {'$ne': None}}
    
]}


filtro={'name': 1, '_id':0, 'offices.latitude': 1, 'offices.longitude': 1,
         'founded_year': 1, 'category_code':1, 'number_of_employees':1,
         'total_money_raised':1}




data=pd.json_normalize(data=colec.find(query, filtro), 
                       record_path='offices', 
                       meta=['category_code', 'name', 'number_of_employees',
                            'founded_year', 'total_money_raised'])


data.head()

,latitude,longitude,category_code,name,number_of_employees,founded_year,total_money_raised
0,34.090368,-118.393064,web,Geni,18,2006,$16.5M
1,37.387845,-122.055197,web,Plaxo,50,2002,$28.3M
2,34.017606,-118.487267,web,Mahalo,40,2007,$21M
3,37.788482,-122.409173,games_video,Kyte,40,2006,$23.4M
4,40.726040,-73.995722,web,Meetup,75,2002,$18.3M


In [13]:
data.insert(2, 'geo_loc', pd.Series(list(range(len(data)))))

In [15]:
# aqui relleno la columna con una lista de longitud y latitud
data['geo_loc']=data.apply(lambda x: [x['longitude'], x['latitude']], axis=1)


data.head()

,latitude,longitude,geo_loc,category_code,name,number_of_employees,founded_year,total_money_raised
0,34.090368,-118.393064,"[-118.393064, 34.090368]",web,Geni,18,2006,$16.5M
1,37.387845,-122.055197,"[-122.055197, 37.387845]",web,Plaxo,50,2002,$28.3M
2,34.017606,-118.487267,"[-118.487267, 34.017606]",web,Mahalo,40,2007,$21M
3,37.788482,-122.409173,"[-122.409173, 37.788482]",games_video,Kyte,40,2006,$23.4M
4,40.726040,-73.995722,"[-73.995722, 40.72604]",web,Meetup,75,2002,$18.3M


In [18]:
data=data.dropna()

In [19]:
data.tail()

,latitude,longitude,geo_loc,category_code,name,number_of_employees,founded_year,total_money_raised
2257,29.735462,-95.413243,"[-95.413243, 29.735462]",games_video,Fictionaut,2,2008,$0
2258,40.756054,-73.986951,"[-73.986951, 40.756054]",web,Conversition,10,2008,$0
2260,59.617364,16.559539,"[16.5595393, 59.617364]",web,Gastrogate,12,1999,$0
2261,49.282455,-123.109217,"[-123.109217, 49.2824551]",games_video,Tiny Speck,45,2009,$17.2M
2262,38.752367,-9.146221,"[-9.1462209, 38.7523672]",games_video,My TV Shows,2,2007,$0


In [20]:
def exchange(col):
    
    res=[]
    
    number=1
    
    symbol=1
    
    for e in col:
        
        if e[-1]=='k': number=1e3
        elif e[-1]=='M': number=1e6
        elif e[-1]=='B': number=1e9
            
        if e[0]=='£': symbol=1.3
        elif e[0]=='€' : symbol=1.12
        
        n_col=re.findall('[0-9.]+', e)[0]

        
        res.append(int(float(n_col) * number * symbol))
        
    return res

In [22]:
exchange(['£28.3M'])

[36790000]

In [24]:
data['n_total_money']=exchange(data.total_money_raised)

data.head()

/var/folders/fv/zc4lh9sn7bnf55fy6jhl0k900000gn/T/ipykernel_84977/862762386.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['n_total_money']=exchange(data.total_money_raised)


,latitude,longitude,geo_loc,category_code,name,number_of_employees,founded_year,total_money_raised,n_total_money
0,34.090368,-118.393064,"[-118.393064, 34.090368]",web,Geni,18,2006,$16.5M,16500000
1,37.387845,-122.055197,"[-122.055197, 37.387845]",web,Plaxo,50,2002,$28.3M,28300000
2,34.017606,-118.487267,"[-118.487267, 34.017606]",web,Mahalo,40,2007,$21M,21000000
3,37.788482,-122.409173,"[-122.409173, 37.788482]",games_video,Kyte,40,2006,$23.4M,23400000
4,40.726040,-73.995722,"[-73.995722, 40.72604]",web,Meetup,75,2002,$18.3M,18300000


In [28]:
data.reset_index(drop=True, inplace=True)

In [29]:
def classifying(date, employees):
    
    company_class=[]
    
    for i in range(len(date)):
        
        if date[i]>=2008 and employees[i]<100:
            company_class.append('StartUp')
        else:
            company_class.append('BigCorp')
            
    
    return company_class

In [30]:
data['company_class']=classifying(data.founded_year, data.number_of_employees)

/var/folders/fv/zc4lh9sn7bnf55fy6jhl0k900000gn/T/ipykernel_84977/3705065726.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['company_class']=classifying(data.founded_year, data.number_of_employees)


In [31]:
data.head()

,latitude,longitude,geo_loc,category_code,name,number_of_employees,founded_year,total_money_raised,n_total_money,company_class
0,34.090368,-118.393064,"[-118.393064, 34.090368]",web,Geni,18,2006,$16.5M,16500000,BigCorp
1,37.387845,-122.055197,"[-122.055197, 37.387845]",web,Plaxo,50,2002,$28.3M,28300000,BigCorp
2,34.017606,-118.487267,"[-118.487267, 34.017606]",web,Mahalo,40,2007,$21M,21000000,BigCorp
3,37.788482,-122.409173,"[-122.409173, 37.788482]",games_video,Kyte,40,2006,$23.4M,23400000,BigCorp
4,40.726040,-73.995722,"[-73.995722, 40.72604]",web,Meetup,75,2002,$18.3M,18300000,BigCorp


In [ ]:
# insertar en mongo

#df.to_json('geoloc.json', orient='records', lines=True)

In [32]:
db.geoloc.insert_many(data.to_dict('records'))

In [33]:
db.geoloc.create_index([('geo_loc', '2dsphere')])

'geo_loc_2dsphere'

In [34]:
# geolocalizacion

def geoquery(df):
    
    list_money_mean=[]
    
    sb_ratio=[]
    
    n_companies=[]
    
    for i in range(len(df)):
        
        q=list(db.geoloc.find({'geo_loc': {'$near': {'$geometry': {
                                     
                                'type': 'Point',
                                'coordinates': df.geo_loc[i]},
                                
                                '$maxDistance': 1000}}}))
        
        
        long_list=len(q)
        
        # media de dinero
        money_mean=sum([q[j]['n_total_money'] for j in range(long_list)]) / long_list
        
        list_money_mean.append(money_mean)
        
        # startup-bigcorp
        
        big=sum(q[k]['company_class'].count('BigCorp') 
                if q[k]['company_class'].count('BigCorp')!=0 
                else 1 for k in range(long_list))
        
        start=sum(q[k]['company_class'].count('StartUp')  for k in range(long_list))
        
        # si sb_ratio=1 entonces big=start, si sb_ratio<1 hay más big, si sb_ratio>1 hay mas start
        sb_ratio.append(start/big)
        
        
        # nº compañias
        n_companies.append(long_list)
        
        
    df['around_companies']=n_companies
    
    df['sb_ratio']=sb_ratio
    
    df['money_density']=list_money_mean
    
    return df

In [35]:
data=geoquery(data)

data.head()

/var/folders/fv/zc4lh9sn7bnf55fy6jhl0k900000gn/T/ipykernel_84977/3181126508.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['around_companies']=n_companies
/var/folders/fv/zc4lh9sn7bnf55fy6jhl0k900000gn/T/ipykernel_84977/3181126508.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sb_ratio']=sb_ratio
/var/folders/fv/zc4lh9sn7bnf55fy6jhl0k900000gn/T/ipykernel_84977/3181126508.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

,latitude,longitude,geo_loc,category_code,name,number_of_employees,founded_year,total_money_raised,n_total_money,company_class,around_companies,sb_ratio,money_density
0,34.090368,-118.393064,"[-118.393064, 34.090368]",web,Geni,18,2006,$16.5M,16500000,BigCorp,1,0.000000,1.650000e+07
1,37.387845,-122.055197,"[-122.055197, 37.387845]",web,Plaxo,50,2002,$28.3M,28300000,BigCorp,2,0.000000,1.415000e+07
2,34.017606,-118.487267,"[-118.487267, 34.017606]",web,Mahalo,40,2007,$21M,21000000,BigCorp,3,0.333333,8.333333e+06
3,37.788482,-122.409173,"[-122.409173, 37.788482]",games_video,Kyte,40,2006,$23.4M,23400000,BigCorp,24,0.375000,8.157333e+06
4,40.726040,-73.995722,"[-73.995722, 40.72604]",web,Meetup,75,2002,$18.3M,18300000,BigCorp,16,0.187500,4.370625e+06


In [51]:
def geomap(df, zoom_start=1.5):

    df.index=range(len(df)) 

    mapa=folium.Map(location=[0, 0],
                    tiles='stamentoner', 
                    zoom_start=zoom_start)
    
    
    for i in range(1, len(df)):

        folium.CircleMarker([float(df.latitude[i]), float(df.longitude[i])], 
                            popup=str(df.name[i]), 
                            radius=1, 
                            icon=folium.Icon()).add_to(mapa)

    #mapa.save('images/geomap.html')
    
    return mapa

In [52]:
print('oficinas')

mapa=geomap(data.loc[:1500])

mapa

oficinas


In [39]:
print ('Compañias alrededor>=10, sb_ratio>0.45 y densidad monetaria>1.000.000$')

df=data[(data.around_companies >= 10) & (data.sb_ratio > 0.45) & (data.money_density > 1e6)]

mapa=geomap(df)

mapa

Compañias alrededor>=10, sb_ratio>0.45 y densidad monetaria>1.000.000$


In [48]:
print ('Compañias alrededor>=10, sb_ratio>0.5 y densidad monetaria>100.000$')

df=data[(data.around_companies >= 10) & (data.sb_ratio > 0.5) & (data.money_density > 1e5)]

mapa=geomap(df)

mapa

Compañias alrededor>=10, sb_ratio>0.5 y densidad monetaria>100.000$
